# Walker Thermostat – Minimal Interactive Model

Questo notebook implementa la forma classica del **termostato di Walker** (Walker, Hays & Kasting, 1981) in versione minimale:

$$\frac{W}{W_0} = \left(\frac{P}{P_0}\right)^{\beta} \exp\!\left(\frac{\Delta T}{T_*}\right)$$

con accoppiamento climatico logaritmico:

$$T = T_0 + \Lambda\,\ln\!\left(\frac{P}{P_0}\right), \quad \Delta T = T - T_0 = \Lambda\,\ln\!\left(\frac{P}{P_0}\right).$$

Allo stato stazionario $F = W/W_0 = F_{out}/W_0$ e ne segue

$$\left(\frac{P}{P_0}\right)^{\beta + \Lambda/T_*} = F \;\Rightarrow\; \frac{P}{P_0} = F^{\;1/(\beta + \Lambda/T_*)}, \quad \Delta T = \Lambda\,\ln\!\left(F^{\;1/(\beta + \Lambda/T_*)}\right).$$

Parametri controllabili:
- **F** = $F_{out}/W_0$ (rapporto del degassamento al riferimento)
- **β** (0.2–0.6 tipico)
- **T\*** in K (10–20 tipico)
- **Sensibilità climatica per raddoppio** $S_{2x}$ in K; si usa $\Lambda = S_{2x}/\ln 2$.

Il notebook permette di variare questi parametri e visualizzare:\
1) gli stati di equilibrio $(P/P_0, \Delta T)$;\
2) la dipendenza di equilibrio al variare di F in un intervallo.


In [0]:
#@title Interattivo: calcolo equilibrio e grafico
import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

LN2 = math.log(2.0)

def walker_equilibrium(F=1.0, beta=0.3, Tstar=13.7, S2x=3.0, F_min=0.5, F_max=1.5):
    Lambda = S2x / LN2
    expo = beta + (Lambda / Tstar)
    if expo <= 0:
        raise ValueError("beta + Lambda/T* deve essere > 0 per avere un equilibrio fisico.")
    P_ratio = F ** (1.0 / expo)
    dT = Lambda * math.log(P_ratio)

    Fs = np.linspace(F_min, F_max, 200)
    P_eq = Fs ** (1.0 / expo)
    dT_eq = Lambda * np.log(P_eq)

    print(f"Parametri: beta={beta:.3f}, T*={Tstar:.2f} K, S2x={S2x:.2f} K, Lambda={Lambda:.3f} K")
    print(f"Equilibrio: P/P0 = {P_ratio:.3f},  ΔT = {dT:.3f} K  (F = {F:.3f})")

    plt.figure(figsize=(6,4))
    plt.plot(Fs, dT_eq)
    plt.xlabel('F = F_out / W0')
    plt.ylabel('ΔT eq (K)')
    plt.title('Walker thermostat: ΔT di equilibrio vs F')
    # Evidenzia il punto corrente
    plt.scatter([F], [dT], s=30)
    plt.grid(True, which='both', linestyle=':')
    plt.show()

    return P_ratio, dT

interact(
    walker_equilibrium,
    F=FloatSlider(value=1.0, min=0.5, max=1.5, step=0.01, description='F'),
    beta=FloatSlider(value=0.3, min=0.2, max=0.6, step=0.01, description='beta'),
    Tstar=FloatSlider(value=13.7, min=8.0, max=25.0, step=0.1, description='T* (K)'),
    S2x=FloatSlider(value=3.0, min=1.5, max=6.0, step=0.1, description='S2x (K)'),
    F_min=FloatSlider(value=0.5, min=0.1, max=0.9, step=0.01, description='F min'),
    F_max=FloatSlider(value=1.5, min=1.1, max=3.0, step=0.01, description='F max'),
)


## Note tecniche

La stabilità lineare locale dell'equilibrio richiede $\beta + \Lambda/T_* > 0$. Con parametri fisicamente ragionevoli (β > 0, T*>0, Λ>0) è soddisfatta. L'output ΔT è riferito allo stato $T_0$ e adotta una risposta logaritmica con sensibilità per raddoppio $S_{2x}$.
